<a href="https://colab.research.google.com/github/BernardBernardes/py_dss_interface_colab_integration/blob/main/py_dss_interface_google_colab_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📦 Instalação do py_dss_interface no Google Colab

Nesta etapa, vamos clonar o repositório da biblioteca `py_dss_interface` diretamente do GitHub,
compilar o OpenDSS para Linux (necessário no ambiente do Google Colab) e instalar a biblioteca no modo "editável".
Isso permite usar a interface do OpenDSS diretamente em Python, o que será útil para simular circuitos e acessar seus resultados via código.

⚠️ Esta instalação é necessária apenas no Google Colab. Em computadores com OpenDSS instalado no Windows, isso não é necessário.

🔧 Etapas:
- Clona o repositório do py_dss_interface
- Compila a versão do OpenDSS para Linux (via script bash)
- Instala o pacote no modo editável



In [ ]:
# Passo 1. Clona e instala py_dss_interface em modo editável
!git clone https://github.com/PauloRadatz/py_dss_interface.git
%cd py_dss_interface
!bash OpenDSSLinuxCPPForRepo.sh
!pip install -e .

🔁 Reinício obrigatório do ambiente

Após instalar a biblioteca `py_dss_interface`, o ambiente do Google Colab precisa ser reiniciado
para que as alterações (especialmente as bibliotecas nativas em C++) sejam carregadas corretamente.
Esta reinicialização será feita automaticamente pelo comando abaixo.

⚠️ Após rodar esta célula, o Colab será reiniciado automaticamente e apresentará uma mensagem de interrompimento de execução.
Será necessário reexecutar as células a partir do passo 3.

In [ ]:
# Passo 2. Reinicia o ambiente (obrigatório após instalar pacotes com bindings nativos)
import os
os.kill(os.getpid(), 9)

📁 Clonar exemplos de circuitos em OpenDSS

Agora vamos clonar o repositório com exemplos de circuitos em OpenDSS utilizados nos testes da biblioteca `py_dss_interface`.
Esses exemplos incluem circuitos clássicos (como o IEEE 13 nós), com arquivos `.dss` já preparados para simulação.

Usaremos esses arquivos para testar a interface com o OpenDSS, explorar os comandos e extrair dados como tensões, potências e configurações de carga.

✅ Após clonar, os arquivos estarão disponíveis nos caminho:

/content/EPRI-OpenDSS-Examples

In [ ]:
# Passo 3. Clona exemplos
!git clone https://github.com/BernardBernardes/EPRI-OpenDSS-Examples.git

📌 Definir o arquivo Master do circuito OpenDSS

Nesta etapa, você deve indicar o caminho completo até o arquivo `.dss` principal do circuito,
também conhecido como "arquivo Master". É esse arquivo que o OpenDSS irá processar inicialmente.

Ele geralmente contém comandos `Redirect` para os demais arquivos que compõem o circuito (cargas, linhas, loadshapes etc). modifique o perâmetro file_full_path com o caminho e o nome do arquivo desejado (basta navegar até o arquivo, nas opções à esquerda, clicar nos três pontos a direta do nome do arquivo e copiar o caminho. Por exemplo, para abrir o circuito IEEE 13 Barras exportado no py-dss-interface-examples:

**file_full_path = "/content/py-dss-interface-examples/feeders/13Bus/IEEE13Nodeckt.dss"**

✅ Se quiser usar outro circuito, basta substituir o caminho acima pelo novo arquivo Master desejado.

In [ ]:
# 🔧🔧🔧 ATENÇÃO: MODIFIQUE AQUI 🔧🔧🔧
# ======================================
# Esta é a célula onde você indica o caminho e nome do arquivo Master.
# ======================================

# Passo 4. Caminho e Arquivo DSS do Master

file_full_path = "/content/Caso_Test/Master.dss"

📄 Leitura completa do circuito via arquivo Master (.dss)

O OpenDSS utiliza a instrução `Redirect` nos arquivos `.dss` para incluir outros arquivos auxiliares
(linhas, cargas, loadshapes, etc.). No ambiente do Google Colab, precisamos simular esse comportamento manualmente.

A função abaixo lê o arquivo Master `.dss` linha por linha. Sempre que encontra um `Redirect`,
ela abre o arquivo referenciado e também copia seu conteúdo.

✅ Resultado: toda a definição do circuito (com arquivos incluídos) será carregada como uma única string Python.

📄 Leitura e execução completa do circuito DSS no Python

Agora que temos o caminho do arquivo Master (.dss), vamos criar uma função que:

🔁 1. Lê o conteúdo desse arquivo linha por linha;

🔍 2. Verifica se há comandos `Redirect` ou `Compile` que apontam para outros arquivos;

📂 3. Em caso positivo, abre esses arquivos também, de forma recursiva;

🔗 4. Junta tudo em uma única string com o circuito expandido.

Após isso, usamos o `py_dss_interface` para:

⚙️ - Instanciar o OpenDSS no Python

💡 - Executar cada linha do circuito usando `dss.text(...)`

In [ ]:
def ler_dss_expandido(caminho_arquivo , arquivos_processados=None):
    import os

    if arquivos_processados is None:
        arquivos_processados = set()

    conteudo = ""

    # Evita redirecionamento em loop
    caminho_arquivo = os.path.abspath(caminho_arquivo)
    if caminho_arquivo in arquivos_processados:
        print(f"Aviso: Arquivo já processado (evitando ciclo): {caminho_arquivo}")
        return ""  # Não adiciona conteúdo novamente

    arquivos_processados.add(caminho_arquivo)

    with open(caminho_arquivo, 'r') as f:
        for linha in f:
            linha_strip = linha.strip()
            if linha_strip.lower().startswith("redirect") or linha_strip.lower().startswith("compile"):
                # Extrair o caminho do arquivo referenciado
                caminho_referenciado = linha_strip.split(" ", 1)[1].strip().replace('"', '')

                # Montar caminho absoluto
                diretorio_base = os.path.dirname(caminho_arquivo)
                caminho_completo = os.path.join(diretorio_base, caminho_referenciado)

                # Chamada recursiva para expandir arquivos referenciados
                conteudo += ler_dss_expandido(caminho_completo)
            else:
                conteudo += linha

    return conteudo

# Usar a função passando o caminho do arquivo Master
codigo_dss = ler_dss_expandido(file_full_path)
#print(codigo_dss)


import py_dss_interface

# Instancia o OpenDSS
dss = py_dss_interface.DSS()
dss.text("clear")

# lista de comando que podem travar a execução
comandos_proibidos = ["redirect", "compile", "clear", "reset", "show", "plot", "solve", "visualize"]

# Passa cada linha do dss expandido como comando
for linha in codigo_dss.splitlines():
    linha = linha.strip()

    if not linha or any(linha.lower().startswith(cmd) for cmd in comandos_proibidos):
        continue  # pula linhas perigosas

    linha_ascii = linha.encode("ascii", errors="ignore").decode()  # remove os acentos
    #print(linha_ascii)
    dss.text(linha_ascii)

**⚡ Execução do circuito e extração de resultados**

**A partir deste ponto, o circuito já foi totalmente carregado na engine do OpenDSS via py_dss_interface.**
**Agora, resolvemos o circuito com o comando `solve` e extraímos as informações desejadas.**

**🧠 É neste bloco em diante que as alterações no código devem ser feitas pelos alunos, para realizar simulações, modificar parâmetros visualizar resultados ou exportar arquivos.**

**📌 Exemplos comuns de modificações:**
*   **Inserir cargas ou elementos**
*   **Alterar perfis de carga, geração e outros**
*   **Executar diferentes modos de simulação**
*   **Exportar e plotar relatórios para análise externa**
*   **- Etc...**









In [ ]:
from posixpath import basename
# 🔧🔧🔧 ATENÇÃO: MODIFIQUE AQUI 🔧🔧🔧
# ======================================
# Esta é a célula principal onde você pode alterar parâmetros,
# adicionar elementos ou extrair novos resultados.
# ======================================
#dss.text("New monitor.B160 Element=Line.L110 terminal=2 mode=1 ppolar=n")

#dss.text("Set mode=Daily")
#dss.text("Set stepsize=1")
#dss.text("Set Number=24")

print(dss.pvsystems.names)
dss.pvsystems.name = "pv"
dss.pvsystems.pf = 0.8

dss.text("solve")

arquivo_result = dss.text("Export monitor PV_voltage")

#print("Barras: ", dss.circuit.buses_names)
print("Quant Barra: ", dss.circuit.num_buses)
#print("Linhas:", dss.lines.names)
print("Quant Linhas: ", dss.lines.count)
print(" ")
print("Potência total:", dss.circuit.total_power)



arquivo_result = dss.text("Export Result")

## Esse código serve apenas para mover o arquivo gerado com o código Export para
## uma pasta desejada, por exemplo a pasta /content'
import os
import shutil

arquivo_copiado = os.path.basename(arquivo_result)
arquivo_copiado = os.path.join("/content/",arquivo_copiado)
shutil.move(arquivo_result, arquivo_copiado)